###Instalando as Ferramentas Necessárias no Colab
Instalar o AWS CLI, kubectl e eksctl no ambiente do Colab.

Instalar AWS CLI

In [ ]:
# Baixar e instalar o AWS CLI versão 2
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!apt-get install -y unzip
!unzip awscliv2.zip
!sudo ./aws/install

# Verificar a instalação
!aws --version

Instalar kubectl

In [ ]:
# Baixar a versão mais recente do kubectl
!curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"

# Tornar o kubectl executável
!chmod +x kubectl

# Mover o kubectl para um diretório no PATH
!sudo mv kubectl /usr/local/bin/

Instalar eksctl


In [ ]:
# Baixar e instalar o eksctl
!curl --silent --location "https://github.com/weaveworks/eksctl/releases/latest/download/eksctl_Linux_amd64.tar.gz" | tar xz -C /tmp
!sudo mv /tmp/eksctl /usr/local/bin

# Verificar a instalação
!eksctl version

###Configurando as Credenciais AWS no Colab

In [ ]:
import getpass

# Inserir Access Key ID
access_key = getpass.getpass('Insira sua AWS Access Key ID: ')

# Inserir Secret Access Key
secret_key = getpass.getpass('Insira sua AWS Secret Access Key: ')

###Configurar o AWS CLI com as Credenciais


In [ ]:
# Configurar o AWS CLI com as credenciais fornecidas
!aws configure set aws_access_key_id {access_key}
!aws configure set aws_secret_access_key {secret_key}

# Definir a região padrão (substitua conforme necessário)
!aws configure set region us-west-2

# Verificar a configuração
!aws sts get-caller-identity

In [18]:
!chmod 400 chave_4.pem

###Criando o Cluster EKS com eksctl
Criar um cluster EKS chamado web-server-cluster na região us-west-2.

In [ ]:
# Criar o cluster EKS
!eksctl create cluster \
  --name web-server-cluster \
  --version 1.24 \
  --region us-west-2 \
  --nodegroup-name web-server-nodes \
  --node-type t3.medium \
  --nodes 3 \
  --nodes-min 1 \
  --nodes-max 4 \
  --managed \
  --ssh-access \
  --ssh-public-key chave_4.pem

###Configurando o kubectl para Acessar o Cluster

In [ ]:
# Atualizar o contexto do kubectl
!eksctl utils write-kubeconfig --cluster=web-server-cluster --region=us-west-2

Verificar a Conexão com o Cluster

In [ ]:
# Listar os nós do cluster
!kubectl get nodes

###Implantando o Servidor Web (Nginx) no Cluster
Criar um Deployment e um Service para o Nginx.

a. Criar o Deployment e Service

In [ ]:
# Definir o YAML do Deployment e Service
nginx_deployment = """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
spec:
  replicas: 2
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.21
        ports:
        - containerPort: 80
        resources:
          requests:
            cpu: "100m"
          limits:
            cpu: "500m"
---
apiVersion: v1
kind: Service
metadata:
  name: nginx-service
spec:
  selector:
    app: nginx
  ports:
    - protocol: TCP
      port: 80
      targetPort: 80
  type: LoadBalancer
"""

# Salvar o YAML em um arquivo
with open('nginx-deployment.yaml', 'w') as f:
    f.write(nginx_deployment)

# Aplicar o Deployment e Service
!kubectl apply -f nginx-deployment.yaml

Verificar os Pods e Serviços

In [ ]:
# Verificar os pods
!kubectl get pods

# Verificar os serviços
!kubectl get services

###Configurando o Autoescalamento Horizontal (HPA)
O Horizontal Pod Autoscaler (HPA) ajusta automaticamente o número de réplicas de um Deployment com base no uso de CPU ou outras métricas.

a. Instalar o Metrics Server
O HPA depende do Metrics Server para obter métricas de utilização de recursos.

In [ ]:
# Instalar o Metrics Server
!kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml

# Verificar se o Metrics Server está funcionando
!kubectl get deployment metrics-server -n kube-system

Criar o HPA para o Deployment Nginx

In [ ]:
# Criar o HPA
!kubectl autoscale deployment nginx-deployment --cpu-percent=50 --min=2 --max=10

# Verificar o HPA
!kubectl get hpa

###Testando o Autoescalamento
Para verificar se o HPA está funcionando corretamente, é possível gerar carga no servidor Nginx e observar como o número de réplicas aumenta.

a. Gerar Carga no Servidor Nginx

In [ ]:
# Executar um pod temporário para gerar carga
!kubectl run -i --tty load-generator --image=busybox -- /bin/sh -c "while true; do wget -q -O- http://nginx-service; done"

###Monitorar o HPA
Em outra célula, monitore o HPA para ver as réplicas sendo ajustadas.

In [ ]:
!kubectl get hpa nginx-deployment --watch

###Acessando o Servidor Web
Depois de implantado, acessar o servidor web usando o endereço EXTERNAL-IP do serviço nginx-service.

In [ ]:
# Obter o EXTERNAL-IP do serviço
!kubectl get service nginx-service

###Limpeza dos Recursos
Após concluir os testes, é importante destruir o cluster EKS para evitar cobranças indesejadas.



In [ ]:
# Destruir o cluster EKS
!eksctl delete cluster --name web-server-cluster --region us-west-2